1. PeriodO aufrufen: https://perio.do/en/
2. PeriodO client öffnen (im Browser das Speichern permanenter Daten zualssen). Dieser sollte dann in https://client.perio.do/?page=open-backend geöffnet sein.
3. Add data source
4. Create authority in data source
5. Create a Testperiod, that can be deleted later.
6. Adjust and execute the follwoing code.

In [89]:
import pandas as pd
import requests
import json

import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
import os
import shutil


In [90]:
chronontology_url = "https://chronontology.dainst.org/data/period/"

df_data = pd.read_excel("Chronontology_to_PeriodO_testdata.xlsx", sheet_name="Import 2025-05-05")
df_chron_gaz_mapping = pd.read_excel("chronontology_gazetteer_mapping.xlsx", sheet_name="Sheet1")
df_gaz_wikidata_mapping = pd.read_excel("Gazetteer_To_Wikidata_Mapping_2025-07-11.xlsx", sheet_name="wikidata")

#Selenium Setup

periodo_client_url = "https://client.perio.do/?page=authority-view&backendID=local-2&authorityID=https%3A%2F%2Fclient.perio.do%2F.well-known%2Fgenid%2F78f08493643b6b882c4d0f79ce02eb99"

#Selenium Setup

def setup_periodo_profile():
    source_profile = r"C:\Users\lukas\AppData\Roaming\Mozilla\Firefox\Profiles\wwqozyt8.default-release"
    selenium_profile = r"C:\temp\selenium_periodo_profile"
    
    if not os.path.exists(selenium_profile):
        shutil.copytree(source_profile, selenium_profile)
    
    return selenium_profile

print(df_data.head())
#print(df_chron_gaz_mapping.head())
#print(df_gaz_wikidata_mapping.head())


       ChronoID      importID   arabic period names  \
0  iFSXg6L89xVF  spp2143:0001                   NaN   
1  tJdKHsJK6jA1  spp2143:0033     الحضارة الأكسومية   
2  ZqkcclMYyHzW  spp2143:0127  العصر الحديدي المبكر   
3  xxfgFLtn03AI  spp2143:0367                   NaN   

                                               names  \
0                                  Early Holocene@en   
1          Aksumite Culture@en\nالحضارة الأكسومية@ar   
2   Early Iron Age (EIA)@en\nالعصر الحديدي المبكر@ar   
3  Ptolemy VI Philometor and Cleopatra II 2nd rei...   

                       types provenance spatiallyPartOfRegion  \
0                 geological    SPP2143       2042600 (world)   
1   material_culture:pottery    SPP2143               2281810   
2  cultural,material_culture    SPP2143      2353208, 2378988   
3                  political    SPP2143               2359911   

                 hasCoreArea  isNamedAfter      isPartOf  ...  \
0                        NaN           NaN  uDnhWiMl

In [91]:
#Retreive JSON from Chronontology API

def get_period_data(period_id):
    url = f"{chronontology_url}{period_id}"
    response = requests.get(url)
    data = response.json()
    return data
    

In [92]:
#Original Label mappt auf Names

def get_label(data):
    names = data['resource']['names']
    if 'en' in names:
        label = names['en']
    else:
        label = names[0]
    
    label = label[0]
    return label

chronontology_languages = {
    'ar':'http://lexvo.org/id/iso639-1/ar', #ara Arabisch
    'de':'http://lexvo.org/id/iso639-1/de', #deu Deutsch
    'el':'http://lexvo.org/id/iso639-1/el', #ell Neugriechisch
    'en':'http://lexvo.org/id/iso639-1/en', #eng Englisch
    'es':'http://lexvo.org/id/iso639-1/es', #spa Spanisch
    'eu':'http://lexvo.org/id/iso639-1/eu', #eus Baskisch
    'fr':'http://lexvo.org/id/iso639-1/fr', #fra Französisch
    'it':'http://lexvo.org/id/iso639-1/it', #ita Italienisch
    'nl':'http://lexvo.org/id/iso639-1/nl', #nld Niederländisch
    'pl':'http://lexvo.org/id/iso639-1/pl', #pol Polnisch
    'pt':'http://lexvo.org/id/iso639-1/pt', #por Portugiesisch
    'ru':'http://lexvo.org/id/iso639-1/ru', #rus Russisch
    'sq':'http://lexvo.org/id/iso639-1/sq', #sqi Albanisch
    'sr':'http://lexvo.org/id/iso639-1/sr', #srp Serbisch
    'tk':'http://lexvo.org/id/iso639-1/tr', #tuk Türkisch
    'vi':'http://lexvo.org/id/iso639-1/vi', #vie Vietnamesisch
    'zh':'http://lexvo.org/id/iso639-1/zh', #zho Chinesisch
    'la':'http://lexvo.org/id/iso639-1/la', #lat Latein
    'egy': 'http://lexvo.org/id/iso639-3/egy', #egy Ägyptisch
    'grc': 'http://lexvo.org/id/iso639-3/grc', #grc Altgriechisch
    'xmr': 'http://lexvo.org/id/iso639-3/xmr', #Meroitisch
    'cop': 'http://lexvo.org/id/iso639-3/cop'} #Koptisch

def get_label_language(data):
    names = data['resource']['names']
    #get the first key in the names dictionary
    first_key = next(iter(names))
    language = chronontology_languages[first_key]
    return language

def get_language_tag(language):
    language_tag = language.split('/')[-1]
    return language_tag

    

In [93]:
#Alternative Namen mit Sprachangabe

def get_all_names(data):
    names = data['resource']['names']
    return names

In [94]:
#Standardisiere Jahreszahlen
def standardize_date(date_str):
    if date_str is None or date_str == "":
        return None
    if "-" in date_str:
        negative = True
    else:
        negative = False
    # Remove any non-numeric characters
    standardized_date = ''.join(filter(str.isdigit, date_str))
    # If the date is less than 4 digits, pad it with leading zeros
    if len(standardized_date) < 4:
        standardized_date = standardized_date.zfill(4)
    if negative == True:
        standardized_date = "-" + standardized_date
    else:
        pass
    return standardized_date

#test_year = "-0"
#print(f"Standardized year: {standardize_date(test_year)}")

In [95]:
#Start (Originaltext und standardisiert)

def get_startdate(data):
    start_date = data['resource']['hasTimespan'][0]
    if 'begin' in start_date:
        start_date = start_date['begin']
        if "at" in start_date:
            start_date = start_date['at']
        elif 'notBefore' and 'notAfter' in start_date:
            start_date = start_date['notBefore']
        elif 'notBefore' in start_date:
            start_date = start_date['notBefore']
        elif 'notAfter' in start_date:
            start_date = start_date['notAfter']
        else:
            pass
    start_date = standardize_date(start_date)
    return start_date

def get_startdate_label(data):
    start_date = data['resource']['hasTimespan'][0]
    if 'timeOriginal' in start_date:
        start_date_label = start_date['timeOriginal']
    else:
        pass
    return start_date_label


In [96]:
#Ende (Originaltext und standardisiert)

def get_enddate(data):
    end_date = data['resource']['hasTimespan'][0]
    if 'begin' in end_date:
        end_date = end_date['end']
        if "at" in end_date:
            end_date = end_date['at']
        elif 'notBefore' and 'notAfter' in end_date:
            end_date = end_date['notBefore']
        elif 'notBefore' in end_date:
            end_date = end_date['notBefore']
        elif 'notAfter' in end_date:
            end_date = end_date['notAfter']
        else:
            pass
    end_date = standardize_date(end_date)
    return end_date

def get_enddate_label(data):
    end_date = data['resource']['hasTimespan'][0]
    if 'timeOriginal' in end_date:
        end_date_label = end_date['timeOriginal']
    else:
        pass
    return end_date_label

In [97]:
#Merge Description and Definition
def merge_desciption_definition(data):
    if 'description' in data['resource']:
        description = data['resource']['description']
    else:
        description = ""
    if 'definition' in data['resource']:
        definition = data['resource']['definition']
    else:
        definition = ""
    
    if description and definition:
        merged_text = f"{description} {definition}"
    elif description:
        merged_text = description
    elif definition:
        merged_text = definition
    else:
        merged_text = ""
    
    return merged_text

In [98]:
#Literature

def get_locator(data):
    if 'references' in data['resource']:
        literature = data['resource']['references']
        literature = literature[0]['reference']
        return literature
    else:
        pass

In [99]:
#Spatial Coverage mit Wikidata

def get_spatial_description(chronoID):
    # Retrieve the spatial ID from the mapping DataFrame
    spatial_description = df_chron_gaz_mapping.loc[df_chron_gaz_mapping['ChronoID'] == chronoID, 'Localization'].values
    if len(spatial_description) > 0:
        spatial_description = spatial_description[0]
    else:
        spatial_description = None
    return spatial_description

def get_spatial_id(chronoID):
    gaz_uri = df_chron_gaz_mapping.loc[df_chron_gaz_mapping['ChronoID'] == chronoID, 'GazetteerID'].values
    if len(gaz_uri) == 0:
        return None
    spatial_id = df_gaz_wikidata_mapping.loc[df_gaz_wikidata_mapping['GazetteerID'] == gaz_uri[0], 'Wikidata_URL'].values
    if len(spatial_id) > 0:
        spatial_id = spatial_id[0]
    else:
        spatial_id = None
    return spatial_id


In [100]:
#Chronontology URL
def get_chronontology_url(period_id):
    url = f"https://chronontology.dainst.org/period/{period_id}"
    return url

In [101]:
#note

def get_note(data):
    note = data['resource']
    if 'note' in note:
        note = note['note']
    else:
        note = ""
    return note

In [102]:
#Info: PeriodO JSON Format and Mapping
'''
def format_json(label, language, language_tag, names, start_date, start_date_label, end_date, end_date_label, description, locator, spatial_description, spatial_id, web_page, note):
    json_data = {        
            "type":"Period",
            "url": web_page,
            "source":{
                "locator": locator
            },
            "label": label,
            "language": language,
            "language_tag": language_tag,
            "localizedLabels": names,
            "spatialCoverage": [{
                "id": spatial_id,
            }
            ],
            "spatialCoverageDescription": spatial_description,
            "start":{
                "in":{
                    "year": start_date
                },
                "label": start_date_label
            },
            "end":{
                "in":{
                    "year": end_date
                },
                "label": end_date_label
            },
            "note": description,
            "editorialNote": note,
        }
    
    return json_data
'''

'\ndef format_json(label, language, language_tag, names, start_date, start_date_label, end_date, end_date_label, description, locator, spatial_description, spatial_id, web_page, note):\n    json_data = {        \n            "type":"Period",\n            "url": web_page,\n            "source":{\n                "locator": locator\n            },\n            "label": label,\n            "language": language,\n            "language_tag": language_tag,\n            "localizedLabels": names,\n            "spatialCoverage": [{\n                "id": spatial_id,\n            }\n            ],\n            "spatialCoverageDescription": spatial_description,\n            "start":{\n                "in":{\n                    "year": start_date\n                },\n                "label": start_date_label\n            },\n            "end":{\n                "in":{\n                    "year": end_date\n                },\n                "label": end_date_label\n            },\n            "n

In [ ]:
#Starte Selenium und lade PeriodO Client
# Verwende das kopierte Profil
profile_path = setup_periodo_profile()
options = Options()
options.add_argument(f"-profile")
options.add_argument(profile_path)

# Zusätzliche Optionen für bessere Stabilität
options.add_argument("--disable-blink-features=AutomationControlled")
options.set_preference("dom.webdriver.enabled", False)
options.set_preference("useAutomationExtension", False)

driver = webdriver.Firefox(options=options)

driver.get(periodo_client_url)

# Warten bis PeriodO geladen ist
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Warten auf ein spezifisches Element (anpassen je nach PeriodO-Interface)
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )
    print("PeriodO Client erfolgreich geladen")
except:
    print("Warnung: PeriodO Client möglicherweise nicht vollständig geladen")

PeriodO Client erfolgreich geladen


In [104]:
#Set up result DataFrame

chrono_url_list = []
locator_list = []
label_list = []
language_list = []
language_tag_list = []
localized_labels_list = []
spatial_coverage_list = []
spatial_coverage_description_list = []
start_date_list = []
start_date_label_list = []
end_date_list = []
end_date_label_list = []
description_list = []
note_list = []

In [105]:
#Main Loop

for index, row in df_data.iterrows():
    data = get_period_data(row['ChronoID'])
    print(data)
    # Klicke auf den Button "Add Period"
    try:
        add_period_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/div/div[1]/div[3]/a[3]"))
        )
        add_period_button.click()
    except Exception as e:
        print(f"Fehler beim Klicken auf 'Add period': {e}")
        continue
    label = get_label(data)
    label_list.append(label)
    #Insert data into PerioO
    try:
        label_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//input[contains(@class, 'css-174ooto')]"))
        )
        label_input.send_keys(label)
    except Exception as e:
        print(f"Fehler beim Eingeben des Labels: {e}")
        continue
    spatial_description = get_spatial_description(row['ChronoID'])
    spatial_coverage_description_list.append(spatial_description)
    #Insert spatial description into PeriodO
    try:
        spatial_description_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//input[contains(@name, 'description')]"))
        )
        spatial_description_input.send_keys(spatial_description)
    except Exception as e:
        print(f"Fehler beim Eingeben der räumlichen Beschreibung: {e}")
        continue
    description = merge_desciption_definition(data)
    description_list.append(description)
    #Insert description into PeriodO
    try:
        description_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//textarea[contains(@class, 'css-gv8lam')]"))
        )
        description_input.send_keys(description)
    except Exception as e:
        print(f"Fehler beim Eingeben der Beschreibung: {e}")
        continue

    start_date = get_startdate(data)
    start_date_list.append(start_date)
    start_date_label_list.append(start_date)
    #Insert start date into PeriodO
    print(f"Start date: {start_date}")
    try:
        start_date_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//label[contains(text(), 'Start label')]/following-sibling::input"))
        )
        start_date_input.send_keys(start_date)
    except Exception as e:
        print(f"Fehler beim Eingeben des Startdatums: {e}")
        continue

    end_date = get_enddate(data)
    end_date_list.append(end_date)
    end_date_label_list.append(end_date)
    #Insert start date into PeriodO
    print(f"End date: {end_date}")
    try:
        end_date_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//label[contains(text(), 'Stop label')]/following-sibling::input"))
        )
        end_date_input.send_keys(end_date)
    except Exception as e:
        print(f"Fehler beim Eingeben des Enddatums: {e}")
        continue

    #Insert locator into PeriodO
    locator = get_locator(data)
    locator_list.append(locator)
    try:
        locator_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//input[contains(@name, 'locator')]"))
        )
        locator_input.send_keys(locator)
    except Exception as e:
        print(f"Fehler beim Eingeben des Locators: {e}")
        continue

    #Insert URL into PeriodO
    web_page = get_chronontology_url(row['ChronoID'])
    chrono_url_list.append(web_page)
    try:
        url_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//input[contains(@name, 'url')]"))
        )
        url_input.send_keys(web_page)
    except Exception as e:
        print(f"Fehler beim Eingeben der URL: {e}")
        continue

    #Insert note into PeriodO
    note = get_note(data)
    note_list.append(note)
    try:
        note_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//textarea[contains(@name, 'editorial-note')]"))
        )
        note_input.send_keys(note)
    except Exception as e:
        print(f"Fehler beim Eingeben der Notiz: {e}")
        continue


    #Fill in results with function outcomes that can not be handeled by Selenium
    language = get_label_language(data)
    language_list.append(language)
    language_tag = get_language_tag(language)
    language_tag_list.append(language_tag)
    localized_labels = get_all_names(data)
    localized_labels_list.append(localized_labels)
    spatial_id = get_spatial_id(row['ChronoID'])
    spatial_coverage_list.append(spatial_id)
    
    #'''
    #klick save button
    try:
        save_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/div/div[2]/div/div[7]/div[3]/button"))
        )
        save_button.click()
    except Exception as e:
        print(f"Fehler beim Klicken auf den Speichern-Button: {e}")
        continue
    #wait for the save to complete
    time.sleep(2)
    #'''

{'resource': {'externalId': 'spp2143:0001', 'names': {'en': ['Early Holocene']}, 'types': ['geological'], 'provenance': 'SPP2143', 'description': 'The beginning of the Holocene epoch marked by a series of high-amplitude and different climatic changes throughout the world. There was an increase in rainfall, a proliferation of lakes, high solar radiation, and a spread of vegetation, among others. These climatic changes have been recorded through different methods including ice cores, radiocarbon and varve dating.', 'tags': ['aaarc'], 'hasTimespan': [{'timeOriginal': 'https://stratigraphy.org/chart', 'begin': {'at': '-9700', 'isImprecise': True}, 'end': {'at': '-6326', 'isImprecise': True}}], 'references': [{'zenonId': '002059095', 'reference': 'Walker, Mike, Sigfus Johnsen, Sune Olander Rasmussen, Trevor Popp, Jørgen-Peder Steffensen, Phil Gibbard, Wim Hoek, et al. 2009. ‘Formal Definition and Dating of the GSSP (Global Stratotype Section and Point) for the Base of the Holocene Using the

In [106]:
#Check if the save can be done
print(f'Die Liste der Chronontology-URLs ist {len(chrono_url_list)} lang.')
print(f'Die Liste der Locator-Einträge ist {len(locator_list)} lang.')
print(f'Die Liste der Labels ist {len(label_list)} lang.')
print(f'Die Liste der Sprachen ist {len(language_list)} lang.')
print(f'Die Liste der Sprach-Tags ist {len(language_tag_list)} lang.')
print(f'Die Liste der lokalisierten Labels ist {len(localized_labels_list)} lang.')
print(f'Die Liste der räumlichen Abdeckungen ist {len(spatial_coverage_list)} lang.')
print(f'Die Liste der räumlichen Beschreibungen ist {len(spatial_coverage_description_list)} lang.')
print(f'Die Liste der Startdaten ist {len(start_date_list)} lang.')
print(f'Die Liste der Startdaten-Labels ist {len(start_date_label_list)} lang.')
print(f'Die Liste der Enddaten ist {len(end_date_list)} lang.')
print(f'Die Liste der Enddaten-Labels ist {len(end_date_label_list)} lang.')
print(f'Die Liste der Beschreibungen ist {len(description_list)} lang.')
print(f'Die Liste der Notizen ist {len(note_list)} lang.')

#print(spatial_coverage_description_list)

Die Liste der Chronontology-URLs ist 4 lang.
Die Liste der Locator-Einträge ist 4 lang.
Die Liste der Labels ist 4 lang.
Die Liste der Sprachen ist 4 lang.
Die Liste der Sprach-Tags ist 4 lang.
Die Liste der lokalisierten Labels ist 4 lang.
Die Liste der räumlichen Abdeckungen ist 4 lang.
Die Liste der räumlichen Beschreibungen ist 4 lang.
Die Liste der Startdaten ist 4 lang.
Die Liste der Startdaten-Labels ist 4 lang.
Die Liste der Enddaten ist 4 lang.
Die Liste der Enddaten-Labels ist 4 lang.
Die Liste der Beschreibungen ist 4 lang.
Die Liste der Notizen ist 4 lang.


In [ ]:
#'''
#Save data as Backup file

#Klick on Settings
try:
    settings_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/div/div[1]/div[2]/a[7]"))
    )
    settings_button.click()
except Exception as e:
    print(f"Fehler beim Klicken auf die Einstellungen: {e}")

#Klick on Backup
try:
    backup_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/div/div[2]/div[2]/button"))
    )
    backup_button.click()
except Exception as e:
    print(f"Fehler beim Klicken auf den Backup-Button: {e}")

#Wait for the backup to complete
time.sleep(5)

#Close the driver
driver.quit()
#Delete the copied profile
if os.path.exists(profile_path):
    shutil.rmtree(profile_path)
#'''




#Create DataFrame with results
result_df = pd.DataFrame({
    'Chronontology URL': chrono_url_list,
    'Locator': locator_list,
    'Label': label_list,
    'Language': language_list,
    'Language Tag': language_tag_list,
    'Localized Labels': localized_labels_list,
    'Spatial Coverage': spatial_coverage_list,
    'Spatial Coverage Description': spatial_coverage_description_list,
    'Start Date': start_date_list,
    'Start Date Label': start_date_label_list,
    'End Date': end_date_list,
    'End Date Label': end_date_label_list,
    'Description': description_list,
    'Note': note_list
})

#Save the DataFrame as CSV
result_df.to_csv("Chronontology_to_PeriodO_results.csv", index=False)

In [ ]:
#Get the most recent file from the Downloads folder
downloads_folder = os.path.join(os.path.expanduser("~"), "Downloads")
recent_file = max([os.path.join(downloads_folder, f) for f in os.listdir(downloads_folder)], key=os.path.getctime)

#get the current working directory in Jupyter
script_directory = os.getcwd()

#Copy the most recent file to the current directory
shutil.copy(recent_file, script_directory)

'k:\\FAIR.rdm AAArC 5\\Systeme\\PeriodO\\periodo-backup_FAIR.rdm Testdata LABEL_2025-07-22(1).json'

In [115]:
print(recent_file)

#get the filename of the most recent file
recent_filename = os.path.basename(recent_file)

print(f"Most recent file: {recent_filename}")

C:\Users\lukas\Downloads\periodo-backup_FAIR.rdm Testdata LABEL_2025-07-22(1).json
Most recent file: periodo-backup_FAIR.rdm Testdata LABEL_2025-07-22(1).json


Nicht alle Informationen könnten mit Selenium eingegeben werden. Deswegen muss das Backup-File nun erneut eingelesen werden, um die nächsten Daten zu ergänzen.

Dazu muss die Backup-Datei vom Downloads-Ordner manuell ins Arbeitsverzeichnis verschoben oder kopiert werden. Der Dateiname im nächsten Codeblock muss angepasst werden.

In [116]:
#read in json data
with open(f'{recent_filename}', 'r', encoding='utf-8') as file:
    data = json.load(file)

#read in results of data mapping

df_additional_data = pd.read_csv("Chronontology_to_PeriodO_results.csv")

print(data)
print(df_additional_data)

{'backend': {'label': 'FAIR.rdm Testdata LABEL', 'description': 'This is the FAIR.rdm Testdataset DESCRIPTION', 'created': 1748417582847, 'modified': 1753174836656, 'accessed': 1753174836717}, 'patches': [{'forward': [{'op': 'add', 'path': '/authorities/https:~1~1client.perio.do~1.well-known~1genid~178f08493643b6b882c4d0f79ce02eb99', 'value': {'id': 'https://client.perio.do/.well-known/genid/78f08493643b6b882c4d0f79ce02eb99', 'type': 'Authority', 'periods': {}, 'source': {'title': 'Entangled Africa (SPP 2143) TITLE', 'url': 'https://www.dainst.blog/entangled-africa/en/home/', 'yearPublished': '2025', 'creators': [{'name': 'Lukas Lammers'}, {'name': 'AAArC'}], 'contributors': [{'name': 'Eymard Fäder'}, {'name': 'University of Cologne'}], 'citation': 'Entangeld Africa Periods, provides by AAArC at the University of Cologne, 2025.', 'locator': 'Main Locator'}, 'editorialNote': 'Der SPP 2143 ist super und seine Perioden sind sehr wertvoll. Für die volle Experience muss man aber in iDAI.chr

In [117]:
#get the authority id, necesary to access the periods

authority_id = data['dataset']['authorities']
authority_id = authority_id.keys()  # Get the first authority ID
authority_id = list(authority_id)[0]  # Convert to list and get the first element
print(authority_id)

https://client.perio.do/.well-known/genid/78f08493643b6b882c4d0f79ce02eb99


In [ ]:
import ast

#Main Loop for adding additional data to the periods

for idx, row in df_additional_data.iterrows():
    period_id = row['genId']
    print(f"Processing period with ID: {period_id}")
    period_data = periods_dict[period_id]

    # Hinzufügen von mehrsprachigen Namen
    # Localized labels are stored as string, need to convert to dict
    localized_labels_str = row['Localized Labels']
    if isinstance(localized_labels_str, str):
        try:
            localized_labels = ast.literal_eval(localized_labels_str)
        except Exception as e:
            print(f"Fehler beim Parsen von Localized Labels: {e}")
            localized_labels = {}
    else:
        localized_labels = localized_labels_str
    print(f"Localized labels: {localized_labels}")
    #print(type(localized_labels))
    period_data['localizedLabels'] = localized_labels

    # Hinzufügen von räumlicher Verortung mit Wikidata
    wiki_id = row['Spatial Coverage']
    wiki_label = row['Spatial Coverage Description']
    spatial_coverage = [{"id": f"{wiki_id}", "label": f"{wiki_label}"}]
    print(f"Spatial coverage: {spatial_coverage}")
    period_data['spatialCoverage'] = spatial_coverage


Processing period with ID: https://client.perio.do/.well-known/genid/33609639821066a00aa897e392f6eafb
Localized labels: {'en': ['Early Holocene']}
<class 'dict'>
Spatial coverage: [{'id': 'https://www.wikidata.org/wiki/Q2', 'label': 'Welt'}]
Processing period with ID: https://client.perio.do/.well-known/genid/75a37cb7f32b6883f2723d045bb4b76a
Localized labels: {'en': ['Aksumite Culture'], 'ar': ['الحضارة الأكسومية']}
<class 'dict'>
Spatial coverage: [{'id': 'https://www.wikidata.org/wiki/Q200127', 'label': 'Tigray'}]
Processing period with ID: https://client.perio.do/.well-known/genid/7702173f6e3af20b96acb2a15dfaccf4
Localized labels: {'en': ['Early Iron Age (EIA)'], 'ar': ['العصر الحديدي المبكر']}
<class 'dict'>
Spatial coverage: [{'id': 'https://www.wikidata.org/wiki/Q575245', 'label': 'Lake Chad Basin'}]
Processing period with ID: https://client.perio.do/.well-known/genid/f5717db013bea36e84b37ce93446872f
Localized labels: {'en': ['Ptolemy VI Philometor and Cleopatra II 2nd reign'], '

In [131]:
#write the modified data back to a new json file
with open('modified_periodo_backup.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

Die soeben gespeichtere JSON-Datei kann im PeriodO Client hochgeladen werden. Auf der Hauptseite unter Restore from backup/Choose backup file.

In [ ]:
#Has part / is part of übernehmen
# Muss wahrscheinlich in separatem Teil geschehen, wenn die genIDs vorliegen.